<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/dynamic_external_unit_operation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic simulation with a custom external unit operation
This tutorial shows how to extend NeqSim with a Python-based unit operation that participates in dynamic process simulations. We implement an electrically heated coil with thermal inertia and connect it to a simple gas process so that students can study the interaction between the custom model and the built-in equipment.


## Learning objectives
- Build a minimal dynamic flowsheet in NeqSim.
- Implement a custom unit operation in Python that follows the `unitop` interface.
- Derive and integrate a first-order energy balance to drive dynamic behaviour.
- Log and visualise results from a transient simulation.


## 0. Environment setup
The notebook is designed for Google Colab, where `neqsim` and `matplotlib` are available. Run the next cell if you need to install or upgrade NeqSim in a fresh environment.


In [ ]:
%%capture
!pip install neqsim

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
from neqsim import jneqsim
from neqsim.process.unitop import unitop


## 1. Define the base process
We start by creating a feed gas, assigning composition and flow conditions, and routing the stream into the flowsheet. The stream is set to *dynamic* mode so that NeqSim integrates mass and energy balances over time.


In [ ]:
# Define a lean gas mixture using the SRK equation of state
feed_fluid = jneqsim.thermo.system.SystemSrkEos(273.15 + 15.0, 50.0)
feed_fluid.addComponent('methane', 0.70)
feed_fluid.addComponent('ethane', 0.20)
feed_fluid.addComponent('n-butane', 0.10)
feed_fluid.setMixingRule('classic')

# Build the inlet stream and enable dynamic calculations
feed_stream = jneqsim.process.equipment.stream.Stream('feed gas', feed_fluid)
feed_stream.setFlowRate(2.0, 'kg/sec')
feed_stream.setPressure(50.0, 'bara')
feed_stream.setTemperature(280.0)  # Kelvin by default
feed_stream.setCalculateSteadyState(False)

# Display the initial thermodynamic state
print(f'Feed temperature: {feed_stream.getTemperature('C'):.1f} °C')
print(f'Feed pressure: {feed_stream.getPressure('bara'):.1f} bara')
print(f'Mass flow rate: {feed_stream.getFlowRate('kg/hr'):.1f} kg/h')


## 2. Implement a dynamic external heater
To represent equipment that is not bundled with NeqSim we extend the `unitop` base class.
The heater stores energy in a thermal mass and exchanges heat with the flowing gas.
Its state variable is the wall temperature $T$.

### 2.1 Energy balance formulation
Applying the first law of thermodynamics to a lumped heater wall gives

$$
  C \frac{dT}{dt} = \dot m c_p (T_{\text{in}} - T) + Q_{\text{heater}} - U A (T - T_{\text{amb}}),
$$
where $C$ is the wall heat capacity [kJ/K], $\dot m$ the mass flow rate [kg/s], $c_p$ the flowing-fluid heat capacity [kJ/kg K], $Q_{\text{heater}}$ the applied electric duty [kW], and $U A$ the overall heat-transfer coefficient to the ambient [kW/K].
The three terms on the right-hand side respectively describe convective exchange with the stream, deliberate heating, and ambient heat loss.

Dividing by $C$ isolates the time derivative that drives the state update,

$$
  \frac{dT}{dt} = \frac{\dot m c_p}{C}(T_{\text{in}} - T) + \frac{Q_{\text{heater}}}{C} - \frac{U A}{C} (T - T_{\text{amb}}).
$$
NeqSim advances the state explicitly, so over a time step $\Delta t$ the wall temperature becomes

$$
  T_{k+1} = T_k + \Delta t \left[ \frac{\dot m c_p}{C}(T_{\text{in}} - T_k) + \frac{Q_{\text{heater}}}{C} - \frac{U A}{C} (T_k - T_{\text{amb}}) \right].
$$
The larger the thermal mass $C$, the slower the wall responds to disturbances.

### 2.2 Parameter interpretation
- **Thermal mass ($C$):** captures the combined metal and holdup inertia; higher values increase lag and smooth the temperature response.
- **Heater duty ($Q_{\text{heater}}$):** a user-specified input that can vary with time to execute operating scenarios.
- **Heat loss ($U A$):** penalises deviations from the ambient and ensures the model cools down realistically.
- **Stream coupling ($\dot m c_p$):** represents the ability of the process stream to remove or supply energy.


### 2.3 Visualise the heater concept
The following schematic emphasises the energy flows captured by the differential equation: convection from the stream, deliberate electrical heating, and losses to the surroundings.


In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))
ax.axis('off')
# Draw heater body
heater_box = patches.FancyBboxPatch((0.3, 0.2), 0.4, 0.6, boxstyle='round,pad=0.1', linewidth=2, edgecolor='tab:red', facecolor='mistyrose')
ax.add_patch(heater_box)
ax.text(0.5, 0.5, 'Dynamic
Heater', ha='center', va='center', fontsize=12, weight='bold')
# Incoming stream arrow
ax.annotate('', xy=(0.3, 0.5), xytext=(0.05, 0.5), arrowprops=dict(arrowstyle='-|>', linewidth=2, color='tab:blue'))
ax.text(0.07, 0.55, '$T_{\mathrm{in}}$', color='tab:blue', fontsize=11)
# Outgoing stream arrow
ax.annotate('', xy=(0.7, 0.5), xytext=(0.95, 0.5), arrowprops=dict(arrowstyle='-|>', linewidth=2, color='tab:orange'))
ax.text(0.9, 0.55, '$T_{\mathrm{out}}$', color='tab:orange', fontsize=11, ha='right')
# Heater power arrow
ax.annotate('Electrical heating $Q_{\mathrm{heater}}$', xy=(0.5, 0.82), xytext=(0.5, 0.95), ha='center', arrowprops=dict(arrowstyle='-|>', linewidth=2, color='tab:red'))
# Heat loss arrow
ax.annotate('Ambient loss $U A (T - T_{\mathrm{amb}})$', xy=(0.5, 0.18), xytext=(0.5, 0.02), ha='center', arrowprops=dict(arrowstyle='-|>', linewidth=2, color='tab:green'))
ax.set_title('Energy interactions represented by the dynamic heater model', fontsize=12)
plt.tight_layout()


In [ ]:
class DynamicHeater(unitop):
    """Electrically heated coil with thermal inertia.

    Attributes
    ----------
    thermal_mass : float
        Lumped heat capacity of metal and fluid holdup [kJ/K].
    heater_power : float
        External heat input applied to the coil [kW].
    heat_loss_coeff : float
        Overall heat-transfer coefficient to ambient [kW/K].
    ambient_temperature : float
        Ambient temperature that drives heat losses [K].
    """
    def __init__(self, name):
        super().__init__()
        self.setName(name)
        self.inlet = None
        self.outlet = None
        self.thermal_mass = 5000.0
        self.heater_power = 0.0
        self.heat_loss_coeff = 0.0
        self.ambient_temperature = 298.15
        self.temperature = None
        self.last_energy_terms = {
            'inlet_temperature': None,
            'wall_temperature': None,
            'convective': 0.0,
            'heat_input': 0.0,
            'heat_loss': 0.0,
            'storage_rate': 0.0,
            'residual': 0.0,
            'time_step': 0.0,
            'capacity': self.thermal_mass,
        }

    def setInputStream(self, stream):
        """Attach the incoming stream and prepare the outlet clone."""
        self.inlet = stream
        outlet_name = f"{self.getName()} outlet"
        self.outlet = jneqsim.process.equipment.stream.Stream(outlet_name, stream.getFluid().clone())
        self.outlet.setCalculateSteadyState(False)

    def getOutputStream(self):
        return self.outlet

    def setThermalMass(self, value_kJ_per_K):
        self.thermal_mass = float(value_kJ_per_K)

    def setHeaterPower(self, power_kW):
        self.heater_power = float(power_kW)

    def setHeatLoss(self, UA_kW_per_K, ambient_K):
        self.heat_loss_coeff = float(UA_kW_per_K)
        self.ambient_temperature = float(ambient_K)

    def _update_outlet(self):
        if self.temperature is None:
            self.temperature = self.inlet.getTemperature('K')
        updated_fluid = self.inlet.getFluid().clone()
        updated_fluid.setTemperature(self.temperature, 'K')
        updated_fluid.setPressure(self.inlet.getPressure('bara'), 'bara')
        updated_fluid.initProperties()
        self.outlet.setThermoSystem(updated_fluid)
        self.outlet.setPressure(self.inlet.getPressure('bara'))
        self.outlet.setTemperature(self.temperature)

    def _record_energy_terms(self, inlet_temperature, convective, heat_input, heat_loss, storage_rate, residual, dt):
        self.last_energy_terms = {
            'inlet_temperature': float(inlet_temperature),
            'wall_temperature': float(self.temperature),
            'convective': float(convective),
            'heat_input': float(heat_input),
            'heat_loss': float(heat_loss),
            'storage_rate': float(storage_rate),
            'residual': float(residual),
            'time_step': float(dt),
            'capacity': float(self.thermal_mass),
        }

    def getLastEnergyTerms(self):
        """Return a copy of the most recent energy balance terms."""
        return dict(self.last_energy_terms)

    def run(self, uuid):
        """Initialise the state during the steady-state solve."""
        self.temperature = self.inlet.getTemperature('K')
        fluid = self.inlet.getFluid()
        mdot = self.inlet.getFlowRate('kg/hr') / 3600.0
        cp = fluid.getCp('kJ/kgK')
        convective_term = mdot * cp * (self.inlet.getTemperature('K') - self.temperature)
        heat_input = self.heater_power
        heat_loss = self.heat_loss_coeff * (self.temperature - self.ambient_temperature)
        storage_rate = 0.0
        residual = convective_term + heat_input - heat_loss - storage_rate
        self._record_energy_terms(self.temperature, convective_term, heat_input, heat_loss, storage_rate, residual, 0.0)
        self._update_outlet()

    def runTransient(self, dt, uuid):
        """Advance the heater temperature using the first-order energy balance."""
        dt = float(dt)
        fluid = self.inlet.getFluid()
        tin = self.inlet.getTemperature('K')
        if self.temperature is None:
            self.temperature = tin
        previous_temp = self.temperature
        mdot = self.inlet.getFlowRate('kg/hr') / 3600.0  # kg/s
        cp = fluid.getCp('kJ/kgK')
        convective_term = mdot * cp * (tin - previous_temp)
        heat_input = self.heater_power
        heat_loss = self.heat_loss_coeff * (previous_temp - self.ambient_temperature)
        capacity = max(self.thermal_mass, 1e-6)
        rhs = convective_term + heat_input - heat_loss
        self.temperature = previous_temp + rhs * dt / capacity
        storage_rate = capacity * (self.temperature - previous_temp) / dt if dt > 0 else 0.0
        residual = rhs - storage_rate
        self._record_energy_terms(tin, convective_term, heat_input, heat_loss, storage_rate, residual, dt)
        self._update_outlet()


## 3. Assemble the flowsheet
The dynamic heater is inserted between the feed stream and a downstream throttle valve that maintains 45 bara. The `ProcessSystem` collects all units, solves the steady state, and exposes the time-integration settings used in the transient study. The custom class also provides `getLastEnergyTerms()` so that any process controller or historian can audit the first-law balance while the simulation runs.


In [ ]:
# Instantiate the custom heater and configure its physics
heater = DynamicHeater('electric heater')
heater.setInputStream(feed_stream)
heater.setThermalMass(5000.0)       # kJ/K
heater.setHeatLoss(2.0, 295.0)      # kW/K and ambient temperature in K
heater.setHeaterPower(100.0)        # initial heater load in kW

# Add an export valve to close the flowsheet
export_valve = jneqsim.process.equipment.valve.ThrottlingValve('export valve', heater.getOutputStream())
export_valve.setOutletPressure(45.0)
export_valve.setCalculateSteadyState(False)

# Build the process model
process = jneqsim.process.processmodel.ProcessSystem()
process.add(feed_stream)
process.add(heater)
process.add(export_valve)
process.setTimeStep(5.0)  # seconds
process.run()

print(f"Steady heater outlet temperature: {heater.getOutputStream().getTemperature('C'):.2f} °C")


## 4. Execute a transient scenario
We apply three operating phases to illustrate the effect of the external unit:

1. **0–1500 s** – warm-up at 100 kW.
2. **1500–4500 s** – power increased to 300 kW to reach a higher outlet temperature.
3. **4500–9000 s** – power reduced to 50 kW, allowing the heater to cool while gas still flows.

During integration we log time, the heater wall and outlet temperatures, every energy-balance contribution, and the downstream pressure so the mathematics can be checked against the simulated trajectory.


In [ ]:
log_rows = []

def record_state():
    terms = heater.getLastEnergyTerms()
    wall_temp = terms.get('wall_temperature')
    inlet_temp = terms.get('inlet_temperature')
    log_rows.append({
        'time [s]': process.getTime(),
        'heater inlet T [C]': (inlet_temp - 273.15) if inlet_temp is not None else np.nan,
        'heater wall T [C]': (wall_temp - 273.15) if wall_temp is not None else np.nan,
        'heater outlet T [C]': heater.getOutputStream().getTemperature('C'),
        'heater power [kW]': terms.get('heat_input', heater.heater_power),
        'convective term [kW]': terms.get('convective', 0.0),
        'heat loss [kW]': terms.get('heat_loss', 0.0),
        'storage rate [kW]': terms.get('storage_rate', 0.0),
        'energy residual [kW]': terms.get('residual', 0.0),
        'time step [s]': terms.get('time_step', np.nan),
        'thermal mass [kJ/K]': terms.get('capacity', heater.thermal_mass),
        'export pressure [bara]': export_valve.getOutletStream().getPressure('bara')
    })

record_state()
total_steps = 1800
for step in range(total_steps):
    if step == 300:
        heater.setHeaterPower(300.0)
    if step == 900:
        heater.setHeaterPower(50.0)
    process.runTransient()
    record_state()

df = pd.DataFrame(log_rows)
df.head()


## 5. Visualise and verify the response
The plots contrast the heater temperatures with the power schedule and decompose the energy balance into convection, applied duty, and losses. We also quantify the numerical residual to show that the simulated trajectory honours the governing equation.


In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
ax1.plot(df['time [s]'], df['heater outlet T [C]'], color='tab:red', label='Outlet T [°C]')
ax1.plot(df['time [s]'], df['heater wall T [C]'], color='tab:orange', linestyle='--', linewidth=2, label='Wall T [°C]')
ax1.plot(df['time [s]'], df['heater inlet T [C]'], color='tab:green', linestyle=':', linewidth=2, label='Inlet T [°C]')
ax1.set_xlabel('Time [s]')
ax1.set_ylabel('Temperature [°C]')
ax1.legend(loc='upper left')
ax1.grid(True, which='major', linestyle='--', alpha=0.3)

ax2 = ax1.twinx()
ax2.step(df['time [s]'], df['heater power [kW]'], where='post', color='tab:blue', label='Heater power [kW]')
ax2.set_ylabel('Power [kW]', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.legend(loc='upper right')

ax1.set_title('Dynamic heater response driven by a custom unit operation')
plt.tight_layout()
plt.show()

df.tail()


### 5.1 Energy balance decomposition
Tracking every term in the governing equation reveals how the heater stores and exchanges energy. The next figure overlays the convective exchange with the feed, the applied electrical duty, the heat loss to ambient, and the implied accumulation term $C\,dT/dt$.


In [ ]:
fig, (ax_top, ax_bottom) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
ax_top.plot(df['time [s]'], df['heater power [kW]'], label='Electrical duty $Q_{\mathrm{heater}}$', color='tab:red')
ax_top.plot(df['time [s]'], df['convective term [kW]'], label='Stream convection $\dot m c_p (T_\mathrm{in}-T)$', color='tab:blue')
ax_top.plot(df['time [s]'], -df['heat loss [kW]'], label='Ambient loss $-U A (T-T_\mathrm{amb})$', color='tab:green')
ax_top.plot(df['time [s]'], df['storage rate [kW]'], label='Accumulation $C\,dT/dt$', color='tab:orange', linestyle='--')
ax_top.set_ylabel('Energy rate [kW]')
ax_top.set_title('Contributions to the heater energy balance')
ax_top.legend(loc='upper right')
ax_top.grid(True, linestyle='--', alpha=0.3)

ax_bottom.plot(df['time [s]'], df['energy residual [kW]'], color='tab:purple')
ax_bottom.axhline(0.0, color='black', linewidth=1, linestyle=':')
ax_bottom.set_xlabel('Time [s]')
ax_bottom.set_ylabel('Residual [kW]')
ax_bottom.set_title('Energy balance residual (should remain near zero)')
ax_bottom.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
max_residual = df['energy residual [kW]'].abs().max()
print(f'Max absolute residual: {max_residual:.6f} kW')


The residual remains close to numerical round-off, confirming that the logged accumulation balances the sum of convective exchange, electrical heating, and heat loss at every time step.


## Key takeaways
- External Python units integrate seamlessly with NeqSim flowsheets when they inherit from `unitop` and implement both `run` and `runTransient`.
- Writing the unit in terms of an explicit energy balance (here, $C\,dT/dt = \dot m c_p (T_{\text{in}} - T) + Q_{\text{heater}} - U A (T - T_{\text{amb}})$) makes the mathematical assumptions transparent and easy to validate.
- Systematic logging of state variables and energy terms enables rich visualisations, residual checks, and, ultimately, confidence that the numerical solution obeys the governing physics.


## References
- W. L. Luyben, *Process Modeling, Simulation, and Control for Chemical Engineers*, 2nd ed., McGraw-Hill, 1990.
- B. W. Bequette, *Process Control: Modeling, Design, and Simulation*, Prentice Hall, 2003.
- R. B. Bird, W. E. Stewart, and E. N. Lightfoot, *Transport Phenomena*, 2nd ed., John Wiley & Sons, 2002.
